In [1]:
ord("G")

71

What word does the following sequence of numbers represent in ASCII:

108, 105, 115, 116

In [3]:
nums = [108, 105, 115, 116]
r_str = ""

for num in nums:
    r_str += chr(num)
    
print(r_str)

list


This week we will download and run a simple version of the Google PageRank Algorithm. Here is an early paper by Larry Page and Sergy Brin, the founders of Google, that describes their early thoughts about the algorithm:

http://infolab.stanford.edu/~backrub/google.html

We will provide you with sample code and lectures that walk through the sample code:

https://www.py4e.com/code3/pagerank.zip

There is not a lot of new code to write - it is mostly looking at the code and making the code work. You will be able to spider some simple content that we provide and then play with the program to spider some other content. Part of the fun of this assignment is when things go wrong and you figure out how to solve a problem when the program wanders into some data that breaks its retrieval and parsing. So you will get used to starting over with a fresh database and running your web crawl.

# Week_2: Peer Grade: Page Rank
First you will spider 100 pages from http://python-data.dr-chuck.net/ run the page rank algorithm and take some screen shots. Then you will reset the spider process and spider 100 pages from any other site on the Internet, run the page rank algorithm, and take some screen shots. You must score 80% or above to successfully complete this assignment.

Page Rank
First you will spider 100 pages from http://python-data.dr-chuck.net/ run the page rank algorithm and take some screen shots. Then you will reset the spider process and spider 100 pages from any other site on the Internet, run the page rank algorithm, and take some screen shots.

Don't take off points for little mistakes. If they seem to have done the assignment give them full credit. Feel free to make suggestions if there are small mistakes. Please keep your comments positive and useful. The course staff will assign the last 30% of the grade and it may take a few days to get graded.

Please Upload Your Submission:

Screenshot of spdump.py output in command line after crawling 100 pages of python-data.dr-chuck.net including the number of rows retrieved (close to 100). You must run sprank.py before spdump.py in order to show the page rankings.
(Please use PNG or JPG files)

Screenshot showing force.html after you open it in a browser to produce the visualization restricted to 25 nodes for python-data.dr-chuck.net. You must show the starting url.
(Please use PNG or JPG files)

Screenshot of spdump.py output in command line after crawling 100 pages of a website of your choosing (other than dr. Chuck's website) including the number of rows retrieved (close to 100). You must run sprank.py before spdump.py in order to show the page rankings.
(Please use PNG or JPG files)

Screenshot of force.html after you open it in a browser to produce the visualization restricted to 25 nodes for your chosen website. You must show the starting url.
(Please use PNG or JPG files)

In [26]:
# spider.py
import sqlite3
import urllib.error
import ssl
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('../../../00_Common_Data_Sets/databases/spiderwiki.sqlite')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS Pages
    (id INTEGER PRIMARY KEY, url TEXT UNIQUE, html TEXT,
     error INTEGER, old_rank REAL, new_rank REAL)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Links
    (from_id INTEGER, to_id INTEGER)''')

cur.execute('''CREATE TABLE IF NOT EXISTS Webs (url TEXT UNIQUE)''')

# Check to see if we are already in progress...
cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
row = cur.fetchone()
if row is not None:
    print("Restarting existing crawl.  Remove spider.sqlite to start a fresh crawl.")
else :
    starturl = input('Enter web url or enter: ')
    if ( len(starturl) < 1 ) :
        #starturl = 'http://python-data.dr-chuck.net/'
        starturl = "https://www.datacamp.com/"
    if ( starturl.endswith('/') ) : starturl = starturl[:-1]
    web = starturl
    if ( starturl.endswith('.htm') or starturl.endswith('.html') ) :
        pos = starturl.rfind('/')
        web = starturl[:pos]

    if ( len(web) > 1 ) :
        cur.execute('INSERT OR IGNORE INTO Webs (url) VALUES ( ? )', ( web, ) )
        cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( starturl, ) )
        conn.commit()

# Get the current webs
cur.execute('''SELECT url FROM Webs''')
webs = list()
for row in cur:
    print("Appending this row", row, "from", cur, "to webs list....")
    webs.append(str(row[0]))
    print("Appended....-------------------------")
print(webs)

many = 0
while True:
    if ( many < 1 ) :
        sval = input('How many pages:')
        if ( len(sval) < 1 ) : break
        many = int(sval)
    many = many - 1

    cur.execute('SELECT id,url FROM Pages WHERE html is NULL and error is NULL ORDER BY RANDOM() LIMIT 1')
    try:
        row = cur.fetchone()
        print("Selecting from Pages db one row: ", row)
        fromid = row[0]
        url = row[1]
        print("row selected......................................")
        print("")
    except:
        print('No unretrieved HTML pages found')
        many = 0
        break
    print("printing from_id and url:.....")
    print(fromid, url, end=' ')

    # If we are retrieving this page, there should be no links from it
    cur.execute('DELETE from Links WHERE from_id=?', (fromid, ) )
    try:
        document = urlopen(url, context=ctx)

        html = document.read()
        if document.getcode() != 200 :
            print("Error on page: ",document.getcode())
            cur.execute('UPDATE Pages SET error=? WHERE url=?', (document.getcode(), url) )

        if 'text/html' != document.info().get_content_type() :
            print("Ignore non text/html page")
            cur.execute('DELETE FROM Pages WHERE url=?', ( url, ) )
            conn.commit()
            continue
        print("HTML length: ")
        print('('+str(len(html))+')', end=' ')
        
        soup = BeautifulSoup(html, "html.parser")
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except:
        print("Unable to retrieve or parse page")
        cur.execute('UPDATE Pages SET error=-1 WHERE url=?', (url, ) )
        conn.commit()
        continue

    cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( url, ) )
    cur.execute('UPDATE Pages SET html=? WHERE url=?', (memoryview(html), url ) )
    conn.commit()

    # Retrieve all of the anchor tags
    tags = soup('a')
    count = 0
    for tag in tags:
        href = tag.get('href', None)
        if ( href is None ) : continue
        # Resolve relative references like href="/contact"
        up = urlparse(href)
        if ( len(up.scheme) < 1 ) :
            href = urljoin(url, href)
        ipos = href.find('#')
        if ( ipos > 1 ) : href = href[:ipos]
        if ( href.endswith('.png') or href.endswith('.jpg') or href.endswith('.gif') ) : continue
        if ( href.endswith('/') ) : href = href[:-1]
        # print href
        if ( len(href) < 1 ) : continue

		# Check if the URL is in any of the webs
        found = False
        for web in webs:
            if ( href.startswith(web) ) :
                found = True
                break
        if not found : continue

        cur.execute('INSERT OR IGNORE INTO Pages (url, html, new_rank) VALUES ( ?, NULL, 1.0 )', ( href, ) )
        count = count + 1
        conn.commit()

        cur.execute('SELECT id FROM Pages WHERE url=? LIMIT 1', ( href, ))
        try:
            row = cur.fetchone()
            toid = row[0]
        except:
            print('Could not retrieve id')
            continue
        # print fromid, toid
        cur.execute('INSERT OR IGNORE INTO Links (from_id, to_id) VALUES ( ?, ? )', ( fromid, toid ) )


    print(count)

cur.close()

Restarting existing crawl.  Remove spider.sqlite to start a fresh crawl.
Appending this row ('https://en.wikipedia.org/wiki/Main_Page',) from <sqlite3.Cursor object at 0x00000232231B1C00> to webs list....
Appended....-------------------------
Appending this row ('https://en.wikipedia.org/wiki/Link_page',) from <sqlite3.Cursor object at 0x00000232231B1C00> to webs list....
Appended....-------------------------
Appending this row ('https://www.datacamp.com',) from <sqlite3.Cursor object at 0x00000232231B1C00> to webs list....
Appended....-------------------------
['https://en.wikipedia.org/wiki/Main_Page', 'https://en.wikipedia.org/wiki/Link_page', 'https://www.datacamp.com']
How many pages:20
Selecting from Pages db one row:  (296, 'https://www.datacamp.com/projects/677')
row selected......................................

printing from_id and url:.....
296 https://www.datacamp.com/projects/677 HTML length: 
(231256) 36
Selecting from Pages db one row:  (424, 'https://www.datacamp.com/u

204 https://www.datacamp.com/courses?q=topic%3Acase_studies HTML length: 
(195703) 14
Selecting from Pages db one row:  (835, 'https://www.datacamp.com/courses/tech:git')
row selected......................................

printing from_id and url:.....
835 https://www.datacamp.com/courses/tech:git HTML length: 
(156982) 113
Selecting from Pages db one row:  (248, 'https://www.datacamp.com/projects/400')
row selected......................................

printing from_id and url:.....
248 https://www.datacamp.com/projects/400 HTML length: 
(228298) 33
Selecting from Pages db one row:  (541, 'https://www.datacamp.com/courses/intermediate-r-practice')
row selected......................................

printing from_id and url:.....
541 https://www.datacamp.com/courses/intermediate-r-practice HTML length: 
(246514) 91
Selecting from Pages db one row:  (415, 'https://www.datacamp.com/users/sign_in?redirect=%2Fprojects%2F697')
row selected......................................

printing f

In [27]:
# sprank.py
import sqlite3

conn = sqlite3.connect('../../../00_Common_Data_Sets/databases/spiderwiki.sqlite')
cur = conn.cursor()

# Find the ids that send out page rank - we only are interested
# in pages in the SCC that have in and out links
cur.execute('''SELECT DISTINCT from_id FROM Links''')
from_ids = list()
for row in cur: 
    from_ids.append(row[0])

# Find the ids that receive page rank 
to_ids = list()
links = list()
cur.execute('''SELECT DISTINCT from_id, to_id FROM Links''')
for row in cur:
    from_id = row[0]
    to_id = row[1]
    if from_id == to_id : continue
    if from_id not in from_ids : continue
    if to_id not in from_ids : continue
    links.append(row)
    if to_id not in to_ids : to_ids.append(to_id)

# Get latest page ranks for strongly connected component
prev_ranks = dict()
for node in from_ids:
    cur.execute('''SELECT new_rank FROM Pages WHERE id = ?''', (node, ))
    row = cur.fetchone()
    prev_ranks[node] = row[0]

sval = input('How many iterations:')
many = 1
if ( len(sval) > 0 ) : many = int(sval)

# Sanity check
if len(prev_ranks) < 1 : 
    print("Nothing to page rank.  Check data.")
    quit()

# Lets do Page Rank in memory so it is really fast
for i in range(many):
    # print prev_ranks.items()[:5]
    next_ranks = dict();
    total = 0.0
    for (node, old_rank) in list(prev_ranks.items()):
        total = total + old_rank
        next_ranks[node] = 0.0
    # print total

    # Find the number of outbound links and sent the page rank down each
    for (node, old_rank) in list(prev_ranks.items()):
        # print node, old_rank
        give_ids = list()
        for (from_id, to_id) in links:
            if from_id != node : continue
           #  print '   ',from_id,to_id

            if to_id not in to_ids: continue
            give_ids.append(to_id)
        if ( len(give_ids) < 1 ) : continue
        amount = old_rank / len(give_ids)
        # print node, old_rank,amount, give_ids
    
        for id in give_ids:
            next_ranks[id] = next_ranks[id] + amount
    
    newtot = 0
    for (node, next_rank) in list(next_ranks.items()):
        newtot = newtot + next_rank
    evap = (total - newtot) / len(next_ranks)

    # print newtot, evap
    for node in next_ranks:
        next_ranks[node] = next_ranks[node] + evap

    newtot = 0
    for (node, next_rank) in list(next_ranks.items()):
        newtot = newtot + next_rank

    # Compute the per-page average change from old rank to new rank
    # As indication of convergence of the algorithm
    totdiff = 0
    for (node, old_rank) in list(prev_ranks.items()):
        new_rank = next_ranks[node]
        diff = abs(old_rank-new_rank)
        totdiff = totdiff + diff

    avediff = totdiff / len(prev_ranks)
    print(i+1, avediff)

    # rotate
    prev_ranks = next_ranks

# Put the final ranks back into the database
print(list(next_ranks.items())[:5])
cur.execute('''UPDATE Pages SET old_rank=new_rank''')
for (id, new_rank) in list(next_ranks.items()) :
    cur.execute('''UPDATE Pages SET new_rank=? WHERE id=?''', (new_rank, id))
conn.commit()
cur.close()

How many iterations:1000
1 0.6052577577432077
2 0.1579396960274718
3 0.0443021544147483
4 0.01190402110016778
5 0.002677458004207795
6 0.0007064645168561963
7 0.00017906305713067744
8 4.7913309718442835e-05
9 1.5446903759650122e-05
10 4.797525967057797e-06
11 1.9503557066761987e-06
12 6.48660120916252e-07
13 2.45652086222866e-07
14 8.657921825355344e-08
15 3.208044338899058e-08
16 1.1954791534046319e-08
17 4.33517836233724e-09
18 1.6272064165205743e-09
19 5.919860230561875e-10
20 2.2079068526701566e-10
21 8.090458964619424e-11
22 2.997499874446872e-11
23 1.1038427116805138e-11
24 4.075927733082176e-12
25 1.5048833777759352e-12
26 5.534829963963551e-13
27 2.0549328961805732e-13
28 7.514850419947463e-14
29 2.771642042859972e-14
30 1.0971948972953715e-14
31 4.557649609189841e-15
32 2.487097829271891e-15
33 1.7573102836844235e-15
34 1.8452501431604723e-15
35 1.0598098362620755e-15
36 9.608951956114796e-16
37 8.417303135678483e-16
38 6.878709620046808e-16
39 6.121095693803876e-16
40 5.34082

317 4.298573968430692e-16
318 3.417759272873489e-16
319 3.5806108644958013e-16
320 2.595004709981632e-16
321 2.285586685899238e-16
322 4.1590880399541897e-16
323 3.215964909341493e-16
324 1.9556352002644659e-16
325 1.0217167248304215e-16
326 3.7675361696622816e-16
327 4.394868822607364e-16
328 3.644335400348011e-16
329 3.747710758508261e-16
330 1.7241027200014388e-16
331 5.140445892078212e-16
332 6.606818267077382e-16
333 2.9525701611523616e-16
334 2.52207551895077e-16
335 2.927080346811478e-16
336 1.4762850805761808e-16
337 3.5671579069270017e-16
338 4.298573968430692e-16
339 3.417759272873489e-16
340 3.5806108644958013e-16
341 2.595004709981632e-16
342 2.285586685899238e-16
343 4.1590880399541897e-16
344 3.215964909341493e-16
345 1.9556352002644659e-16
346 1.0217167248304215e-16
347 3.7675361696622816e-16
348 4.394868822607364e-16
349 3.644335400348011e-16
350 3.747710758508261e-16
351 1.7241027200014388e-16
352 5.140445892078212e-16
353 6.606818267077382e-16
354 2.9525701611523616e-

628 2.52207551895077e-16
629 2.927080346811478e-16
630 1.4762850805761808e-16
631 3.5671579069270017e-16
632 4.298573968430692e-16
633 3.417759272873489e-16
634 3.5806108644958013e-16
635 2.595004709981632e-16
636 2.285586685899238e-16
637 4.1590880399541897e-16
638 3.215964909341493e-16
639 1.9556352002644659e-16
640 1.0217167248304215e-16
641 3.7675361696622816e-16
642 4.394868822607364e-16
643 3.644335400348011e-16
644 3.747710758508261e-16
645 1.7241027200014388e-16
646 5.140445892078212e-16
647 6.606818267077382e-16
648 2.9525701611523616e-16
649 2.52207551895077e-16
650 2.927080346811478e-16
651 1.4762850805761808e-16
652 3.5671579069270017e-16
653 4.298573968430692e-16
654 3.417759272873489e-16
655 3.5806108644958013e-16
656 2.595004709981632e-16
657 2.285586685899238e-16
658 4.1590880399541897e-16
659 3.215964909341493e-16
660 1.9556352002644659e-16
661 1.0217167248304215e-16
662 3.7675361696622816e-16
663 4.394868822607364e-16
664 3.644335400348011e-16
665 3.747710758508261e-1

943 2.52207551895077e-16
944 2.927080346811478e-16
945 1.4762850805761808e-16
946 3.5671579069270017e-16
947 4.298573968430692e-16
948 3.417759272873489e-16
949 3.5806108644958013e-16
950 2.595004709981632e-16
951 2.285586685899238e-16
952 4.1590880399541897e-16
953 3.215964909341493e-16
954 1.9556352002644659e-16
955 1.0217167248304215e-16
956 3.7675361696622816e-16
957 4.394868822607364e-16
958 3.644335400348011e-16
959 3.747710758508261e-16
960 1.7241027200014388e-16
961 5.140445892078212e-16
962 6.606818267077382e-16
963 2.9525701611523616e-16
964 2.52207551895077e-16
965 2.927080346811478e-16
966 1.4762850805761808e-16
967 3.5671579069270017e-16
968 4.298573968430692e-16
969 3.417759272873489e-16
970 3.5806108644958013e-16
971 2.595004709981632e-16
972 2.285586685899238e-16
973 4.1590880399541897e-16
974 3.215964909341493e-16
975 1.9556352002644659e-16
976 1.0217167248304215e-16
977 3.7675361696622816e-16
978 4.394868822607364e-16
979 3.644335400348011e-16
980 3.747710758508261e-1

In [29]:
#spdump.py
import sqlite3

conn = sqlite3.connect('../../../00_Common_Data_Sets/databases/spider.sqlite')
cur = conn.cursor()

cur.execute('''SELECT COUNT(from_id) AS inbound, old_rank, new_rank, id, url 
     FROM Pages JOIN Links ON Pages.id = Links.to_id
     WHERE html IS NOT NULL
     GROUP BY id ORDER BY inbound DESC''')

count = 0
for row in cur :
    if count < 50 : print(row)
    count = count + 1
print(count, 'rows.')
cur.close()

(5, 1.3955081488507628, 1.3955081488507617, 1174, 'http://python-data.dr-chuck.net/known_by_Seona.html')
(5, 2.3307455697751056, 2.330745569775104, 651, 'http://python-data.dr-chuck.net/known_by_Rhiona.html')
(4, 1.16562746556567, 1.1656274655656689, 3327, 'http://python-data.dr-chuck.net/known_by_Mael.html')
(4, 1.9102750495916023, 1.9102750495916008, 3123, 'http://python-data.dr-chuck.net/known_by_Eubh.html')
(4, 1.9016413596992023, 1.9016413596992003, 2614, 'http://python-data.dr-chuck.net/known_by_Meenal.html')
(4, 1.2572650043119535, 1.2572650043119524, 2181, 'http://python-data.dr-chuck.net/known_by_Rhea.html')
(4, 2.4375719977328227, 2.4375719977328205, 2004, 'http://python-data.dr-chuck.net/known_by_Vince.html')
(4, 3.091771088594743, 3.0917710885947414, 1758, 'http://python-data.dr-chuck.net/known_by_Sineidin.html')
(4, 1.380766734710377, 1.3807667347103758, 1632, 'http://python-data.dr-chuck.net/known_by_Yoji.html')
(4, 2.349344119609087, 2.3493441196090856, 1475, 'http://pyt

In [30]:
# spjson.py
import sqlite3

conn = sqlite3.connect('../../../00_Common_Data_Sets/databases/spiderwiki.sqlite')
cur = conn.cursor()

print("Creating JSON output on spider.js...")
howmany = int(input("How many nodes? "))

cur.execute('''SELECT COUNT(from_id) AS inbound, old_rank, new_rank, id, url 
    FROM Pages JOIN Links ON Pages.id = Links.to_id
    WHERE html IS NOT NULL AND ERROR IS NULL
    GROUP BY id ORDER BY id,inbound''')

fhand = open('../../../00_Common_Data_Sets/code3/pagerank/spider.js','w')
nodes = list()
maxrank = None
minrank = None
for row in cur :
    nodes.append(row)
    rank = row[2]
    if maxrank is None or maxrank < rank: maxrank = rank
    if minrank is None or minrank > rank : minrank = rank
    if len(nodes) > howmany : break

if maxrank == minrank or maxrank is None or minrank is None:
    print("Error - please run sprank.py to compute page rank")
    quit()

fhand.write('spiderJson = {"nodes":[\n')
count = 0
map = dict()
ranks = dict()
for row in nodes :
    if count > 0 : fhand.write(',\n')
    # print row
    rank = row[2]
    rank = 19 * ( (rank - minrank) / (maxrank - minrank) ) 
    fhand.write('{'+'"weight":'+str(row[0])+',"rank":'+str(rank)+',')
    fhand.write(' "id":'+str(row[3])+', "url":"'+row[4]+'"}')
    map[row[3]] = count
    ranks[row[3]] = rank
    count = count + 1
fhand.write('],\n')

cur.execute('''SELECT DISTINCT from_id, to_id FROM Links''')
fhand.write('"links":[\n')

count = 0
for row in cur :
    # print row
    if row[0] not in map or row[1] not in map : continue
    if count > 0 : fhand.write(',\n')
    rank = ranks[row[0]]
    srank = 19 * ( (rank - minrank) / (maxrank - minrank) ) 
    fhand.write('{"source":'+str(map[row[0]])+',"target":'+str(map[row[1]])+',"value":3}')
    count = count + 1
fhand.write(']};')
fhand.close()
cur.close()

print("Open force.html in a browser to view the visualization")

Creating JSON output on spider.js...
How many nodes? 30
Open force.html in a browser to view the visualization


# Week_4:
This week we do the first half of a project to download, process, and visualize an email corpus from the Sakai open source project from 2004-2011:

http://mbox.dr-chuck.net/

This is a large amount of data and requires significant cleanup to make sense of the data before we visualize it.

Important: You do not have to download all of the data to complete this project. Depending on your Internet connection, downloading nearly a gigabyte of data might be impossible. All we want to do is to have you download a small subset of the data and run the steps to process the data.

Here is the software we will be using to retrieve and process the email data:

https://www.py4e.com/code3/gmane.zip

If you have a fast network connection with no bandwidth charge - you can download all the data. If you try to download all the data it may take well over 24 hours to pull the data. The good news is that because there are separate crawl, clean, model, and visualization steps, you can start and stop the crawl process as often as you like and run the other processes on the data that has been downloaded so far.

# Assignment_4
Loading and Modeling Mail Data
This assignment you will start the spidering of the email data and run the gmodel.py program and take some screen shots. You are welcome to look at the code and make modifications - but you can complete the assignment without making any code changes. You must score 80% or above to successfully complete this assignment.

Mailing List Data - Part I
In this assignment you will download some of the mailing list data from http://mbox.dr-chuck.net/ and run the data cleaning / modeling process and take some screen shots. All screenshots should use the same data set.

Don't take off points for little mistakes. If they seem to have done the assignment give them full credit. Feel free to make suggestions if there are small mistakes. Please keep your comments positive and useful. The course staff will assign the last 30% of the grade and it make take a few days to get graded.

Please Upload Your Submission:

A screen shot of your SQLiteBrowser showing messages downloaded from mbox.dr-chuck.net into the content.sqlite database including all data fields and the bottom of sqlite browser.
(Please use PNG or JPG files)

A screen shot of you running the gmodel.py application to produce the index.sqlite database. Please include the allsenders line at the beginning of the output. Please do not include code in the screenshot.
(Please use PNG or JPG files)

A screen shot of your SQLiteBrowser showing messages in the index.sqlite database after the gmodel.py has executed including all data fields and the bottom of sqlite browser..
(Please use PNG or JPG files)

A screen shot of you running the gbasic.py program to compute basic histogram data on the messages you have retrieved. Please include the "How many to dump" and "Loaded messages" lines in the output. For a dump number, please use a number less than 25. Do not include code in the screenshot.
(Please use PNG or JPG files)

In [31]:
# gmane.py
import sqlite3
import time
import ssl
import urllib.request, urllib.parse, urllib.error
from urllib.parse import urljoin
from urllib.parse import urlparse
import re
from datetime import datetime, timedelta

# Not all systems have this so conditionally define parser
try:
    import dateutil.parser as parser
except:
    pass

def parsemaildate(md) :
    # See if we have dateutil
    try:
        pdate = parser.parse(tdate)
        test_at = pdate.isoformat()
        return test_at
    except:
        pass

    # Non-dateutil version - we try our best

    pieces = md.split()
    notz = " ".join(pieces[:4]).strip()

    # Try a bunch of format variations - strptime() is *lame*
    dnotz = None
    for form in [ '%d %b %Y %H:%M:%S', '%d %b %Y %H:%M:%S',
        '%d %b %Y %H:%M', '%d %b %Y %H:%M', '%d %b %y %H:%M:%S',
        '%d %b %y %H:%M:%S', '%d %b %y %H:%M', '%d %b %y %H:%M' ] :
        try:
            dnotz = datetime.strptime(notz, form)
            break
        except:
            continue

    if dnotz is None :
        # print 'Bad Date:',md
        return None

    iso = dnotz.isoformat()

    tz = "+0000"
    try:
        tz = pieces[4]
        ival = int(tz) # Only want numeric timezone values
        if tz == '-0000' : tz = '+0000'
        tzh = tz[:3]
        tzm = tz[3:]
        tz = tzh+":"+tzm
    except:
        pass

    return iso+tz

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

conn = sqlite3.connect('../../../00_Common_Data_Sets/databases/content.sqlite')
cur = conn.cursor()

baseurl = "http://mbox.dr-chuck.net/sakai.devel/"

cur.execute('''CREATE TABLE IF NOT EXISTS Messages
    (id INTEGER UNIQUE, email TEXT, sent_at TEXT,
     subject TEXT, headers TEXT, body TEXT)''')

# Pick up where we left off
start = None
cur.execute('SELECT max(id) FROM Messages' )
try:
    row = cur.fetchone()
    if row is None :
        start = 0
    else:
        start = row[0]
except:
    start = 0

if start is None : start = 0

many = 0
count = 0
fail = 0
while True:
    if ( many < 1 ) :
        conn.commit()
        sval = input('How many messages:')
        if ( len(sval) < 1 ) : break
        many = int(sval)

    start = start + 1
    cur.execute('SELECT id FROM Messages WHERE id=?', (start,) )
    try:
        row = cur.fetchone()
        if row is not None : continue
    except:
        row = None

    many = many - 1
    url = baseurl + str(start) + '/' + str(start + 1)

    text = "None"
    try:
        # Open with a timeout of 30 seconds
        document = urllib.request.urlopen(url, None, 30, context=ctx)
        text = document.read().decode()
        if document.getcode() != 200 :
            print("Error code=",document.getcode(), url)
            break
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except Exception as e:
        print("Unable to retrieve or parse page",url)
        print("Error",e)
        fail = fail + 1
        if fail > 5 : break
        continue

    print(url,len(text))
    count = count + 1

    if not text.startswith("From "):
        print(text)
        print("Did not find From ")
        fail = fail + 1
        if fail > 5 : break
        continue

    pos = text.find("\n\n")
    if pos > 0 :
        hdr = text[:pos]
        body = text[pos+2:]
    else:
        print(text)
        print("Could not find break between headers and body")
        fail = fail + 1
        if fail > 5 : break
        continue

    email = None
    x = re.findall('\nFrom: .* <(\S+@\S+)>\n', hdr)
    if len(x) == 1 :
        email = x[0];
        email = email.strip().lower()
        email = email.replace("<","")
    else:
        x = re.findall('\nFrom: (\S+@\S+)\n', hdr)
        if len(x) == 1 :
            email = x[0];
            email = email.strip().lower()
            email = email.replace("<","")

    date = None
    y = re.findall('\Date: .*, (.*)\n', hdr)
    if len(y) == 1 :
        tdate = y[0]
        tdate = tdate[:26]
        try:
            sent_at = parsemaildate(tdate)
        except:
            print(text)
            print("Parse fail",tdate)
            fail = fail + 1
            if fail > 5 : break
            continue

    subject = None
    z = re.findall('\Subject: (.*)\n', hdr)
    if len(z) == 1 : subject = z[0].strip().lower();

    # Reset the fail counter
    fail = 0
    print("   ",email,sent_at,subject)
    cur.execute('''INSERT OR IGNORE INTO Messages (id, email, sent_at, subject, headers, body)
        VALUES ( ?, ?, ?, ?, ?, ? )''', ( start, email, sent_at, subject, hdr, body))
    if count % 50 == 0 : conn.commit()
    if count % 100 == 0 : time.sleep(1)

conn.commit()
cur.close()

How many messages:1
http://mbox.dr-chuck.net/sakai.devel/59643/59644 17553
    matthew@longsight.com 2015-03-20T16:27:12-04:00 re: [building sakai] sakai 10 bulding error
How many messages:1
http://mbox.dr-chuck.net/sakai.devel/59644/59645 13128
    alberto.olivamolina@gmail.com 2015-03-20T16:36:12+01:00 re: [building sakai] sakai 10 bulding error
How many messages:10
http://mbox.dr-chuck.net/sakai.devel/59645/59646 7557
    eric.duquenoy@univ-littoral.fr 2015-03-20T16:52:24+01:00 [building sakai] lti and sakai groups (or sections)
http://mbox.dr-chuck.net/sakai.devel/59646/59647 1


Did not find From 
http://mbox.dr-chuck.net/sakai.devel/59647/59648 9467
    csev@umich.edu 2015-03-20T23:04:53-04:00 re: [building sakai] lti and sakai groups (or sections)
http://mbox.dr-chuck.net/sakai.devel/59648/59649 11727
    matthew@longsight.com 2015-03-20T23:12:47-04:00 re: [building sakai] lti and sakai groups (or sections)
http://mbox.dr-chuck.net/sakai.devel/59649/59650 5916
    blshrestha@mum

http://mbox.dr-chuck.net/sakai.devel/59695/59696 7520
    chimata@e-limu.co.za> 2015-03-26T22:47:07+02:00 [building sakai] sakai 10 version download
http://mbox.dr-chuck.net/sakai.devel/59696/59697 16857
    jeffrey.miller.ctr@usuhs.edu 2015-03-27T08:54:28-04:00 [building sakai] stack trace:
http://mbox.dr-chuck.net/sakai.devel/59697/59698 6641
    plukasew@uwo.ca 2015-03-27T09:19:39-04:00 re: [building sakai]
http://mbox.dr-chuck.net/sakai.devel/59698/59699 8004
    plukasew@uwo.ca 2015-03-27T09:29:08-04:00 re: [building sakai] stack
http://mbox.dr-chuck.net/sakai.devel/59699/59700 8012
    neal.caidin@apereo.org 2015-03-27T09:38:42-04:00 [building sakai] crowd sourcing sakai community survey questions
http://mbox.dr-chuck.net/sakai.devel/59700/59701 18260
    matthew@longsight.com 2015-03-27T10:36:38-04:00 re: [building sakai]
http://mbox.dr-chuck.net/sakai.devel/59701/59702 25820
    mcarro@entornosdeformacion.com 2015-03-27T12:40:40+01:00 [building sakai] smtpfrom@org.sakaiproject.

http://mbox.dr-chuck.net/sakai.devel/59749/59750 10057
    neal.caidin-wgvhjocuptadnm+yrofe0a@public.gmane.org 2015-04-02T08:32:15-04:00 [wg: sakai qa] sakai 11 proposed schedule
http://mbox.dr-chuck.net/sakai.devel/59750/59751 5008
    gaojun@fudan.edu.cn 2015-04-02T08:34:24+08:00 re: [building sakai] sakai 10 casify
http://mbox.dr-chuck.net/sakai.devel/59751/59752 10265
    gaojun@fudan.edu.cn 2015-04-02T08:47:44+08:00 re: [building sakai] new sakai architecture
http://mbox.dr-chuck.net/sakai.devel/59752/59753 18199
    neal.caidin-wgvhjocuptadnm+yrofe0a@public.gmane.org 2015-04-02T08:57:39-04:00 re: [wg: sakai qa] [building sakai] sakai 11 proposed schedule
http://mbox.dr-chuck.net/sakai.devel/59753/59754 6138
    kchan@hawaii.edu 2015-04-02T09:08:09-10:00 [building sakai] announcements tool ... the logic behind modified
http://mbox.dr-chuck.net/sakai.devel/59754/59755 4948
    smit.gsa@gmail.com 2015-04-02T09:25:19+00:00 [building sakai] database connection pooling in sakai
How man

http://mbox.dr-chuck.net/sakai.devel/59802/59803 51899
    jaco@opencollab.co.za 2015-04-10T10:58:33+02:00 fwd: sakai iframe portlet (jsr-168)
http://mbox.dr-chuck.net/sakai.devel/59803/59804 8187
    francois@opencollab.co.za 2015-04-10T11:41:32+02:00 scorm installation instruction for 10.1 & above
http://mbox.dr-chuck.net/sakai.devel/59804/59805 29416
    mcarro@entornosdeformacion.com 2015-04-10T11:45:08+02:00 re: scorm installation instruction for 10.1 & above
http://mbox.dr-chuck.net/sakai.devel/59805/59806 6622
    daniel.merino@unavarra.es 2015-04-10T13:16:01+02:00 new permission's name for sak-11647
http://mbox.dr-chuck.net/sakai.devel/59806/59807 36287
    adam.marshall@it.ox.ac.uk 2015-04-10T13:24:14+00:00 re: indexing in custom search tool
http://mbox.dr-chuck.net/sakai.devel/59807/59808 26709
    mcarro@entornosdeformacion.com 2015-04-10T13:55:48+02:00 re: new permission's name for sak-11647
http://mbox.dr-chuck.net/sakai.devel/59808/59809 31276
    francois@opencollab.co.z

http://mbox.dr-chuck.net/sakai.devel/59857/59858 21227
    anders.nordqvist@his.se 2015-04-15T13:49:20+00:00 re: sakai 2.9.x msgcntr
http://mbox.dr-chuck.net/sakai.devel/59858/59859 9131
    steve.swinsburg@gmail.com 2015-04-15T13:56:57+10:00 re: get sakai site title with jsp
http://mbox.dr-chuck.net/sakai.devel/59859/59860 18639
    steve.swinsburg@gmail.com 2015-04-15T14:00:09+10:00 re: re: sakai upgrade database script
http://mbox.dr-chuck.net/sakai.devel/59860/59861 30900
    alaanizar@iihem.ac.ma 2015-04-15T14:07:55+01:00 re: drop box quota button
http://mbox.dr-chuck.net/sakai.devel/59861/59862 7117
    neal.caidin@apereo.org 2015-04-15T14:51:06-04:00 ambassador to discuss sakai with a college?
http://mbox.dr-chuck.net/sakai.devel/59862/59863 12493
    sakai-core-team-bjrk66m5fvy5gssncpszngm0pvjs0e/a@public.gmane.org 2015-04-15T16:25:07+00:00 [wg: sakai qa] sakai core team call, thursday, 2015-04-16,
http://mbox.dr-chuck.net/sakai.devel/59863/59864 26812
    shaowei.zhang@yale.ed

...

http://mbox.dr-chuck.net/sakai.devel/60184/60185 10903
    adrian.r.fish@gmail.com 2015-05-12T23:41:14+01:00 re: re: clog apereo submission
http://mbox.dr-chuck.net/sakai.devel/60185/60186 8065
    jonas.lindholm@umu.se 2015-05-13T03:38:52-07:00 re: clog apereo submission
http://mbox.dr-chuck.net/sakai.devel/60186/60187 18004
    neal.caidin-wgvhjocuptadnm+yrofe0a@public.gmane.org 2015-05-13T08:20:56-04:00 [sakai-users] open apereo 2015: book your hotel to get the group rate
http://mbox.dr-chuck.net/sakai.devel/60187/60188 17102
    neal.caidin@apereo.org 2015-05-13T08:20:56-04:00 open apereo 2015: book your hotel to get the group rate
http://mbox.dr-chuck.net/sakai.devel/60188/60189 17145
    neal.caidin@apereo.org 2015-05-13T08:20:56-04:00 [sakai-users] open apereo 2015: book your hotel to get the group rate
http://mbox.dr-chuck.net/sakai.devel/60189/60190 17931
    neal.caidin-wgvhjocuptadnm+yrofe0a@public.gmane.org 2015-05-13T08:20:56-04:00 open apereo 2015: book your hotel to get 

http://mbox.dr-chuck.net/sakai.devel/60240/60241 25500
    wilsonk@mail.amc.edu 2015-05-18T19:16:08+00:00 re: disabling some features in profile2
http://mbox.dr-chuck.net/sakai.devel/60241/60242 17709
    dbarnes6891@gmail.com 2015-05-19T05:15:08-07:00 re: having trouble with deployment
http://mbox.dr-chuck.net/sakai.devel/60242/60243 40863
    wilsonk@mail.amc.edu 2015-05-19T07:37:42-07:00 re: disabling some features in profile2
http://mbox.dr-chuck.net/sakai.devel/60243/60244 17903
    jjmerono@um.es 2015-05-19T08:23:35+02:00 re: [sakai-pmc] mathjax for sakai 11?
http://mbox.dr-chuck.net/sakai.devel/60244/60245 9664
    neal.caidin@apereo.org 2015-05-19T08:25:36-04:00 re: [sakai-core] moving morpheus forward
http://mbox.dr-chuck.net/sakai.devel/60245/60246 17649
    adam.marshall@it.ox.ac.uk 2015-05-19T08:55:09+00:00 re: mathjax for sakai 11?
http://mbox.dr-chuck.net/sakai.devel/60246/60247 18506
    ottenhoff@longsight.com 2015-05-19T09:00:05-04:00 re: having trouble with deployment

http://mbox.dr-chuck.net/sakai.devel/60300/60301 5570
    csev@umich.edu 2015-05-22T09:31:01-04:00 do we have a mvn repo that has artifacts for nightly
http://mbox.dr-chuck.net/sakai.devel/60301/60302 8512
    enietzel@anisakai.com 2015-05-22T10:49:28-04:00 re: do we have a mvn repo that has artifacts for nightly
http://mbox.dr-chuck.net/sakai.devel/60302/60303 12021
    adrian.r.fish@gmail.com 2015-05-22T11:08:59+01:00 re: help. please
http://mbox.dr-chuck.net/sakai.devel/60303/60304 11364
    matthew@longsight.com 2015-05-22T13:07:57-04:00 re: do we have a mvn repo that has artifacts for nightly
http://mbox.dr-chuck.net/sakai.devel/60304/60305 7547
    csev@umich.edu 2015-05-22T14:40:17-04:00 re: dumb git question
http://mbox.dr-chuck.net/sakai.devel/60305/60306 11734
    daniel.merino@unavarra.es 2015-05-22T14:52:50+02:00 re: sakai 10.4 question
http://mbox.dr-chuck.net/sakai.devel/60306/60307 10627
    adrian.r.fish@gmail.com 2015-05-22T15:17:41+01:00 re: dumb git question
http://m

http://mbox.dr-chuck.net/sakai.devel/60359/60360 14979
    matthew@longsight.com 2015-05-27T11:27:21-04:00 re: amazon aws and sakai
http://mbox.dr-chuck.net/sakai.devel/60360/60361 8558
    alexandre.balleste@udl.cat 2015-05-27T11:50:28+02:00 peer review period in sakai trunk
http://mbox.dr-chuck.net/sakai.devel/60361/60362 17364
    matthew@longsight.com 2015-05-27T12:22:58-04:00 re: assignment tool behavior change
http://mbox.dr-chuck.net/sakai.devel/60362/60363 6767
    daniel.merino@unavarra.es 2015-05-27T12:30:06+02:00 evalsys migration issues
http://mbox.dr-chuck.net/sakai.devel/60363/60364 16974
    botimer@umich.edu 2015-05-27T12:42:22-04:00 re: dumb git question
http://mbox.dr-chuck.net/sakai.devel/60364/60365 8550
    luke.fernandez@gmail.com 2015-05-27T13:46:35-06:00 re: if you ever wanted to meet roy t. fielding (i.e the
http://mbox.dr-chuck.net/sakai.devel/60365/60366 18801
    neal.caidin@apereo.org 2015-05-27T13:49:02-04:00 re: assignment tool behavior change
http://mbox

http://mbox.dr-chuck.net/sakai.devel/60419/60420 8271
    steve.swinsburg@gmail.com 2015-06-01T21:00:33+10:00 deploying sakai to tomcat /lib by default
http://mbox.dr-chuck.net/sakai.devel/60420/60421 11803
    steve.swinsburg@gmail.com 2015-06-01T22:09:17+10:00 re: morpheus bug, overriding bold/strong/em etc
http://mbox.dr-chuck.net/sakai.devel/60421/60422 1


Did not find From 
http://mbox.dr-chuck.net/sakai.devel/60422/60423 1


Did not find From 
http://mbox.dr-chuck.net/sakai.devel/60423/60424 1


Did not find From 
http://mbox.dr-chuck.net/sakai.devel/60424/60425 1


Did not find From 
http://mbox.dr-chuck.net/sakai.devel/60425/60426 1


Did not find From 
http://mbox.dr-chuck.net/sakai.devel/60426/60427 1


Did not find From 


In [32]:
# gmodel.py
import sqlite3
import time
import re
import zlib
from datetime import datetime, timedelta

# Not all systems have this
try:
    import dateutil.parser as parser
except:
    pass

dnsmapping = dict()
mapping = dict()

def fixsender(sender,allsenders=None) :
    global dnsmapping
    global mapping
    if sender is None : return None
    sender = sender.strip().lower()
    sender = sender.replace('<','').replace('>','')

    # Check if we have a hacked gmane.org from address
    if allsenders is not None and sender.endswith('gmane.org') :
        pieces = sender.split('-')
        realsender = None
        for s in allsenders:
            if s.startswith(pieces[0]) :
                realsender = sender
                sender = s
                # print(realsender, sender)
                break
        if realsender is None :
            for s in mapping:
                if s.startswith(pieces[0]) :
                    realsender = sender
                    sender = mapping[s]
                    # print(realsender, sender)
                    break
        if realsender is None : sender = pieces[0]

    mpieces = sender.split("@")
    if len(mpieces) != 2 : return sender
    dns = mpieces[1]
    x = dns
    pieces = dns.split(".")
    if dns.endswith(".edu") or dns.endswith(".com") or dns.endswith(".org") or dns.endswith(".net") :
        dns = ".".join(pieces[-2:])
    else:
        dns = ".".join(pieces[-3:])
    # if dns != x : print(x,dns)
    # if dns != dnsmapping.get(dns,dns) : print(dns,dnsmapping.get(dns,dns))
    dns = dnsmapping.get(dns,dns)
    return mpieces[0] + '@' + dns

def parsemaildate(md) :
    # See if we have dateutil
    try:
        pdate = parser.parse(md)
        test_at = pdate.isoformat()
        return test_at
    except:
        pass

    # Non-dateutil version - we try our best

    pieces = md.split()
    notz = " ".join(pieces[:4]).strip()

    # Try a bunch of format variations - strptime() is *lame*
    dnotz = None
    for form in [ '%d %b %Y %H:%M:%S', '%d %b %Y %H:%M:%S',
        '%d %b %Y %H:%M', '%d %b %Y %H:%M', '%d %b %y %H:%M:%S',
        '%d %b %y %H:%M:%S', '%d %b %y %H:%M', '%d %b %y %H:%M' ] :
        try:
            dnotz = datetime.strptime(notz, form)
            break
        except:
            continue

    if dnotz is None :
        # print('Bad Date:',md)
        return None

    iso = dnotz.isoformat()

    tz = "+0000"
    try:
        tz = pieces[4]
        ival = int(tz) # Only want numeric timezone values
        if tz == '-0000' : tz = '+0000'
        tzh = tz[:3]
        tzm = tz[3:]
        tz = tzh+":"+tzm
    except:
        pass

    return iso+tz

# Parse out the info...
def parseheader(hdr, allsenders=None):
    if hdr is None or len(hdr) < 1 : return None
    sender = None
    x = re.findall('\nFrom: .* <(\S+@\S+)>\n', hdr)
    if len(x) >= 1 :
        sender = x[0]
    else:
        x = re.findall('\nFrom: (\S+@\S+)\n', hdr)
        if len(x) >= 1 :
            sender = x[0]

    # normalize the domain name of Email addresses
    sender = fixsender(sender, allsenders)

    date = None
    y = re.findall('\nDate: .*, (.*)\n', hdr)
    sent_at = None
    if len(y) >= 1 :
        tdate = y[0]
        tdate = tdate[:26]
        try:
            sent_at = parsemaildate(tdate)
        except Exception as e:
            # print('Date ignored ',tdate, e)
            return None

    subject = None
    z = re.findall('\nSubject: (.*)\n', hdr)
    if len(z) >= 1 : subject = z[0].strip().lower()

    guid = None
    z = re.findall('\nMessage-ID: (.*)\n', hdr)
    if len(z) >= 1 : guid = z[0].strip().lower()

    if sender is None or sent_at is None or subject is None or guid is None :
        return None
    return (guid, sender, subject, sent_at)

conn = sqlite3.connect('../../../00_Common_Data_Sets/databases/index.sqlite')
cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS Messages ''')
cur.execute('''DROP TABLE IF EXISTS Senders ''')
cur.execute('''DROP TABLE IF EXISTS Subjects ''')
cur.execute('''DROP TABLE IF EXISTS Replies ''')

cur.execute('''CREATE TABLE IF NOT EXISTS Messages
    (id INTEGER PRIMARY KEY, guid TEXT UNIQUE, sent_at INTEGER,
     sender_id INTEGER, subject_id INTEGER,
     headers BLOB, body BLOB)''')
cur.execute('''CREATE TABLE IF NOT EXISTS Senders
    (id INTEGER PRIMARY KEY, sender TEXT UNIQUE)''')
cur.execute('''CREATE TABLE IF NOT EXISTS Subjects
    (id INTEGER PRIMARY KEY, subject TEXT UNIQUE)''')
cur.execute('''CREATE TABLE IF NOT EXISTS Replies
    (from_id INTEGER, to_id INTEGER)''')

conn_1 = sqlite3.connect('../../../00_Common_Data_Sets/databases/mapping.sqlite')
cur_1 = conn_1.cursor()

cur_1.execute('''SELECT old,new FROM DNSMapping''')
for message_row in cur_1 :
    dnsmapping[message_row[0].strip().lower()] = message_row[1].strip().lower()

mapping = dict()
cur_1.execute('''SELECT old,new FROM Mapping''')
for message_row in cur_1 :
    old = fixsender(message_row[0])
    new = fixsender(message_row[1])
    mapping[old] = fixsender(new)

# Done with mapping.sqlite
conn_1.close()

# Open the main content (Read only)
conn_1 = sqlite3.connect('file:../../../00_Common_Data_Sets/databases/content.sqlite?mode=ro', uri=True)
cur_1 = conn_1.cursor()

allsenders = list()
cur_1.execute('''SELECT email FROM Messages''')
for message_row in cur_1 :
    sender = fixsender(message_row[0])
    if sender is None : continue
    if 'gmane.org' in sender : continue
    if sender in allsenders: continue
    allsenders.append(sender)

print("Loaded allsenders",len(allsenders),"and mapping",len(mapping),"dns mapping",len(dnsmapping))

cur_1.execute('''SELECT headers, body, sent_at
    FROM Messages ORDER BY sent_at''')

senders = dict()
subjects = dict()
guids = dict()

count = 0

for message_row in cur_1 :
    hdr = message_row[0]
    parsed = parseheader(hdr, allsenders)
    if parsed is None: continue
    (guid, sender, subject, sent_at) = parsed

    # Apply the sender mapping
    sender = mapping.get(sender,sender)

    count = count + 1
    if count % 250 == 1 : print(count,sent_at, sender)
    # print(guid, sender, subject, sent_at)

    if 'gmane.org' in sender:
        print("Error in sender ===", sender)

    sender_id = senders.get(sender,None)
    subject_id = subjects.get(subject,None)
    guid_id = guids.get(guid,None)

    if sender_id is None :
        cur.execute('INSERT OR IGNORE INTO Senders (sender) VALUES ( ? )', ( sender, ) )
        conn.commit()
        cur.execute('SELECT id FROM Senders WHERE sender=? LIMIT 1', ( sender, ))
        try:
            row = cur.fetchone()
            sender_id = row[0]
            senders[sender] = sender_id
        except:
            print('Could not retrieve sender id',sender)
            break
    if subject_id is None :
        cur.execute('INSERT OR IGNORE INTO Subjects (subject) VALUES ( ? )', ( subject, ) )
        conn.commit()
        cur.execute('SELECT id FROM Subjects WHERE subject=? LIMIT 1', ( subject, ))
        try:
            row = cur.fetchone()
            subject_id = row[0]
            subjects[subject] = subject_id
        except:
            print('Could not retrieve subject id',subject)
            break
    # print(sender_id, subject_id)
    cur.execute('INSERT OR IGNORE INTO Messages (guid,sender_id,subject_id,sent_at,headers,body) VALUES ( ?,?,?,datetime(?),?,? )',
            ( guid, sender_id, subject_id, sent_at,
            zlib.compress(message_row[0].encode()), zlib.compress(message_row[1].encode())) )
    conn.commit()
    cur.execute('SELECT id FROM Messages WHERE guid=? LIMIT 1', ( guid, ))
    try:
        row = cur.fetchone()
        message_id = row[0]
        guids[guid] = message_id
    except:
        print('Could not retrieve guid id',guid)
        break

cur.close()
cur_1.close()

Loaded allsenders 1803 and mapping 29 dns mapping 1
1 2005-12-08T23:34:30-06:00 ggolden22@mac.com
251 2005-12-22T10:03:20-08:00 tpamsler@ucdavis.edu
501 2006-01-12T11:17:34-05:00 lance@indiana.edu
751 2006-01-24T11:13:28-08:00 vrajgopalan@ucmerced.edu
1001 2006-02-02T08:27:30-07:00 john.ellis@rsmart.com
1251 2006-02-15T19:13:17-05:00 ggolden22@mac.com
1501 2006-02-23T16:49:00-05:00 csev@umich.edu
1751 2006-03-13T13:48:36-05:00 csev@umich.edu
2001 2006-03-27T13:34:32-06:00 swgithen@mtu.edu
2251 2006-04-05T12:38:43-04:00 ccount@mit.edu
2501 2006-04-17T13:28:54-07:00 jholtzman@berkeley.edu
2751 2006-04-25T14:26:03-07:00 vrajgopalan@ucmerced.edu
3001 2006-05-03T13:49:11-04:00 cheryl.wogahn@yale.edu
3251 2006-05-10T17:14:01+00:00 nuno@ufp.pt
3501 2006-05-17T10:37:49-04:00 ys2n@virginia.edu


C:\Users\buono\Anaconda3\envs\py3spec\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


3751 2006-05-23T14:52:13-04:00 azeckoski@unicon.net
4001 2006-06-06T12:51:32-04:00 ajpoland@indiana.edu
4251 2006-06-14T08:28:01-04:00 lance@indiana.edu
4501 2006-06-21T14:31:06-05:00 mustansar@rice.edu
4751 2006-06-29T15:52:15+01:00 antranig@cam.ac.uk
5001 2006-07-12T11:50:55-04:00 markjnorton@earthlink.net
5251 2006-07-20T17:24:14-07:00 sinou@etudes.org
5501 2006-07-31T18:45:12-04:00 ggolden22@mac.com
5751 2006-08-08T07:44:09-04:00 dave.ross@gmail.com
6001 2006-08-14T15:32:02-04:00 daisy.flemming@gmail.com
6251 2006-08-23T02:02:38-12:00 woleraymond@enownow.com
6501 2006-08-31T13:53:24-04:00 ryaneby@gmail.com
6751 2006-09-12T17:52:18-05:00 zach@aeroplanesoftware.com
7001 2006-09-19T08:34:39+01:00 b.steele@blueyonder.co.uk
7251 2006-09-27T10:56:27+02:00 wilfrid.labeta@gmail.com
7501 2006-10-06T10:03:19+02:00 b.toeter@uva.nl
7751 2006-10-17T10:09:17-07:00 sfischbein@ucdavis.edu
8001 2006-10-27T12:40:07+01:00 nuno@ufp.pt
8251 2006-11-08T10:16:19+00:00 harriet@cam.ac.uk
8501 2006-11-19T17

42251 2011-07-21T08:51:21+10:00 steve.swinsburg@swinsborg.com
42501 2011-08-04T21:40:54-04:00 carl@hallwaytech.com
42751 2011-08-19T17:33:56+00:00 leward@indiana.edu
43001 2011-09-09T10:35:46+02:00 darolmar@upvnet.upv.es
43251 2011-09-28T08:53:32-05:00 yq12@txstate.edu
43501 2011-10-17T09:39:38-07:00 holdorph@unicon.net
43751 2011-11-09T09:52:35-05:00 bkirschn@umich.edu
44001 2011-11-29T10:03:15+02:00 joconnor@uwc.ac.za
44251 2011-12-21T19:11:04+11:00 steve.swinsburg@swinsborg.com
44501 2012-01-19T11:37:49+01:00 diego.delblanco@samoo.es
44751 2012-02-04T11:26:25+00:00 ciellie.jansenvanvuuren@nwu.ac.za
45001 2012-02-15T11:31:22+05:00 jehan.badshah@nu.edu.pk
45251 2012-03-05T11:51:45-07:00 ktakacs@rsmart.com
45501 2012-03-21T10:47:22+01:00 jean-francois.leveque@upmc.fr
45751 2012-04-02T13:33:24+02:00 nonkululeko.phakela@wits.ac.za
46001 2012-04-19T11:06:35-04:00 markjnorton@earthlink.net
46251 2012-05-11T06:46:10-04:00 azeckoski@unicon.net
46501 2012-05-30T09:59:25-05:00 javier@fisica.un

In [34]:
# gbasic.py
import sqlite3
import time
import zlib

howmany = int(input("How many to dump? "))

conn = sqlite3.connect('../../../00_Common_Data_Sets/databases/index.sqlite')
cur = conn.cursor()

cur.execute('SELECT id, sender FROM Senders')
senders = dict()
for message_row in cur :
    senders[message_row[0]] = message_row[1]

cur.execute('SELECT id, subject FROM Subjects')
subjects = dict()
for message_row in cur :
    subjects[message_row[0]] = message_row[1]

# cur.execute('SELECT id, guid,sender_id,subject_id,headers,body FROM Messages')
cur.execute('SELECT id, guid,sender_id,subject_id,sent_at FROM Messages')
messages = dict()
for message_row in cur :
    messages[message_row[0]] = (message_row[1],message_row[2],message_row[3],message_row[4])

print("Loaded messages=",len(messages),"subjects=",len(subjects),"senders=",len(senders))

sendcounts = dict()
sendorgs = dict()
for (message_id, message) in list(messages.items()):
    sender = message[1]
    sendcounts[sender] = sendcounts.get(sender,0) + 1
    pieces = senders[sender].split("@")
    if len(pieces) != 2 : continue
    dns = pieces[1]
    sendorgs[dns] = sendorgs.get(dns,0) + 1

print('')
print('Top',howmany,'Email list participants')

x = sorted(sendcounts, key=sendcounts.get, reverse=True)
for k in x[:howmany]:
    print(senders[k], sendcounts[k])
    if sendcounts[k] < 10 : break

print('')
print('Top',howmany,'Email list organizations')

x = sorted(sendorgs, key=sendorgs.get, reverse=True)
for k in x[:howmany]:
    print(k, sendorgs[k])
    if sendorgs[k] < 10 : break


How many to dump? 5
Loaded messages= 59720 subjects= 29472 senders= 1800

Top 5 Email list participants
steve.swinsburg@swinsborg.com 3337
azeckoski@unicon.net 1907
ian@cam.ac.uk 1591
csev@umich.edu 1490
david.horwitz@uct.ac.za 1221

Top 5 Email list organizations
umich.edu 6805
gmail.com 5683
swinsborg.com 3337
cam.ac.uk 2626
uct.ac.za 2582
